### 1. Tổng quan bộ dataset

- Books data: Cung cấp tổng quan chi tiết về thông tin sách bao gồm tên sách, năm xuất bản cũng như tác giả của cuốn sách
- Users data: Cung cấp tổng quan chi tiết về từng người dùng, chẳng hạn như vị trí của người dùng cũng như độ tuổi của người dùng
- Rating data: Dữ liệu được tổng hợp cơ bản về cách mỗi người dùng xếp hạng từng cuốn sách

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.neighbors import NearestNeighbors
import missingno as msno

### 2. Tiền xử lý data

In [75]:
df_books = pd.read_csv('C:/duong/github/Machine-Learning-self-study/archive/Books.csv', delimiter=",", usecols = [0,1,2,3,4])
print(df_books.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271360 non-null  object
 1   Book-Title           271360 non-null  object
 2   Book-Author          271358 non-null  object
 3   Year-Of-Publication  271360 non-null  object
 4   Publisher            271358 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB
None


C:\Users\kieub\AppData\Local\Temp\ipykernel_2088\2155556344.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_books = pd.read_csv('C:/duong/github/Machine-Learning-self-study/archive/Books.csv', delimiter=",", usecols = [0,1,2,3,4])


Ta có thể thấy cột Year-Of-Publication đang ở dạng object, vì vậy ta cần phải chuyển nó về int

In [76]:
try:
    df_books['Year-Of-Publication']  = df_books['Year-Of-Publication'].astype(int)
except Exception as e:
    print(e)

invalid literal for int() with base 10: 'DK Publishing Inc'


Chotto, có những sách có trường Year-Of-Publication có giá trị là 'DK Publishing Inc'. Vậy ta sẽ in thử những quyển đó là quyển nào

In [70]:
print(df_books[df_books['Year-Of-Publication'] == 'DK Publishing Inc'])

              ISBN                                         Book-Title   
209538  078946697X  DK Readers: Creating the X-Men, How It All Beg...  \
221678  0789466953  DK Readers: Creating the X-Men, How Comic Book...   

       Book-Author Year-Of-Publication   
209538        2000   DK Publishing Inc  \
221678        2000   DK Publishing Inc   

                                                Publisher  
209538  http://images.amazon.com/images/P/078946697X.0...  
221678  http://images.amazon.com/images/P/0789466953.0...  


Giờ ta sẽ chuyển data ở cột Year-Of-Publication sang dạng số, data nào không convert được sẽ ở dạng NaN

In [81]:
df_books['Year-Of-Publication'] = pd.to_numeric(df_books['Year-Of-Publication'],errors='coerce')
print(df_books.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271360 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 271360 non-null  object 
 1   Book-Title           271360 non-null  object 
 2   Book-Author          271358 non-null  object 
 3   Year-Of-Publication  271357 non-null  float64
 4   Publisher            271358 non-null  object 
dtypes: float64(1), object(4)
memory usage: 10.4+ MB
None


Có thể thấy cột Year-Of-Publication tuy convert sang số nhưng lại ở dạng float64, bây giờ chúng ta sẽ loại bỏ các record có value là NaN và ép kiểu cột này thành int32

In [83]:
df_books = df_books.dropna()
df_books['Year-Of-Publication'] = df_books['Year-Of-Publication'].astype(int)
print(df_books.info())

<class 'pandas.core.frame.DataFrame'>
Index: 271353 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 271353 non-null  object
 1   Book-Title           271353 non-null  object
 2   Book-Author          271353 non-null  object
 3   Year-Of-Publication  271353 non-null  int32 
 4   Publisher            271353 non-null  object
dtypes: int32(1), object(4)
memory usage: 11.4+ MB
None


In [73]:
df_books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
